In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scraped-agent-details/agents_full.csv


In [2]:
import pandas as pd
import warnings

# Show all columns and full text
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

warnings.filterwarnings('ignore')

In [3]:
original_df = pd.read_csv("/kaggle/input/scraped-agent-details/agents_full.csv")

In [4]:
for col in original_df.columns:
    print(col)

state
usps
first_year
user_languages
marketing_area_cities.0.city_state
marketing_area_cities.0.name
marketing_area_cities.0.state_code
marketing_area_cities.1.city_state
marketing_area_cities.1.name
marketing_area_cities.1.state_code
marketing_area_cities.2.city_state
marketing_area_cities.2.name
marketing_area_cities.2.state_code
marketing_area_cities.3.city_state
marketing_area_cities.3.name
marketing_area_cities.3.state_code
marketing_area_cities.4.city_state
marketing_area_cities.4.name
marketing_area_cities.4.state_code
marketing_area_cities.5.city_state
marketing_area_cities.5.name
marketing_area_cities.5.state_code
marketing_area_cities.6.city_state
marketing_area_cities.6.name
marketing_area_cities.6.state_code
marketing_area_cities.7.city_state
marketing_area_cities.7.name
marketing_area_cities.7.state_code
marketing_area_cities.8.city_state
marketing_area_cities.8.name
marketing_area_cities.8.state_code
marketing_area_cities.9.city_state
marketing_area_cities.9.name
marketin

In [5]:
df= original_df.copy()

In [6]:
df.shape

(940, 14485)

In [7]:
import re
import pandas as pd


# 1️⃣ Get only marketing_area_cities.N.city_state columns
city_state_cols = [
    c for c in df.columns
    if re.fullmatch(r"marketing_area_cities\.\d+\.city_state", c)
]

# 2️⃣ Combine them into a deduped, ordered list
def to_list(row):
    seen, out = set(), []
    for v in row:
        if pd.notna(v) and v not in seen:
            seen.add(v)
            out.append(v)
    return out

df["marketing_area_cities_as_city_state"] = df[city_state_cols].apply(to_list, axis=1)

# 3️⃣ Convert that list to a comma-separated string for CSV output
df["marketing_area_cities_as_city_state"] = df["marketing_area_cities_as_city_state"].apply(
    lambda lst: ", ".join(lst)
)

# 4️⃣ Drop all the original marketing_area_cities.* columns
drop_cols = [c for c in df.columns if c.startswith("marketing_area_cities.")]
df = df.drop(columns=drop_cols)

In [8]:
df.shape

(940, 4643)

In [9]:


# --- collect served_areas.* indices we can pair (name + state_code) ---
name_cols = {}
state_cols = {}

for c in df.columns:
    m_name = re.fullmatch(r"served_areas\.(\d+)\.name", c)
    m_state = re.fullmatch(r"served_areas\.(\d+)\.state_code", c)
    if m_name:
        name_cols[int(m_name.group(1))] = c
    if m_state:
        state_cols[int(m_state.group(1))] = c

indices = sorted(set(name_cols) | set(state_cols))

# --- combine into "Name_StateCode" per row, dedup, csv-friendly ---
def combine_served_areas(row):
    seen, out = set(), []
    for i in indices:
        nm = row[name_cols[i]] if i in name_cols else None
        sc = row[state_cols[i]] if i in state_cols else None
        if pd.notna(nm) and str(nm).strip() != "":
            part = str(nm).strip()
            if pd.notna(sc) and str(sc).strip() != "":
                part = f"{part}_{str(sc).strip()}"
            if part not in seen:
                seen.add(part)
                out.append(part)
    return ", ".join(out)  # CSV-friendly string

df["served_areas_as_name_state"] = df.apply(combine_served_areas, axis=1)

# --- drop all the original served_areas.* columns ---
drop_served = [c for c in df.columns if c.startswith("served_areas.")]
df = df.drop(columns=drop_served)

# (optional) also drop any stray 'served_areas' root col if present
if "served_areas" in df.columns:
    df = df.drop(columns=["served_areas"])

In [10]:
df.shape

(940, 898)

In [11]:
df.head(5)

,state,usps,first_year,user_languages,role,description,phones.0.ext,phones.0.number,phones.0.type,review_count,office.website,office.address.country,office.address.city,office.address.line,office.address.state,office.address.postal_code,office.address.state_code,office.address.line2,office.phones.0.ext,office.phones.0.number,office.phones.0.type,office.feed_licenses,office.photo.href,office.photo.is_zoomed,office.phone_list.phone_1.ext,office.phone_list.phone_1.number,office.phone_list.phone_1.type,office.video,office.nrds_id,office.fulfillment_id,office.licenses,office.mls.0.member.id,office.mls.0.id,office.mls.0.abbreviation,office.mls.0.type,office.mls.0.primary,office.name,office.slogan,zips,title,has_photo,advertiser_id,mls_monetization,designations,agent_rating,href,id,settings.display_ratings,settings.broker_data_feed_opt_out,settings.cfb_wizard_step,settings.terms_of_use,settings.has_dotrealtor,settings.display_price_range,settings.display_listings,settings.show_stream,settings.full_access,settings.far_override,settings.loaded_from_sb,settings.unsubscribe.account_notify,settings.unsubscribe.autorecs,settings.unsubscribe.recapprove,settings.new_feature_popup_closed.agent_left_nav_avatar_to_profile,settings.reviews_opt_out.rdc,settings.reviews_opt_out.rs,settings.share_contacts,settings.display_sold_listings,settings.profile_wizard.realsatisfied_opt_out,settings.profile_wizard.tt_opt_out,last_updated,types,address.country,address.city,address.line,address.state,address.postal_code,address.state_code,address.line2,specializations.0.name,specializations.1.name,specializations.2.name,specializations.3.name,languages,photo.href,photo.is_zoomed,feed_licenses,person_name,nrds_id,broker.accent_color,broker.designations,broker.name,broker.photo.href,broker.photo.is_zoomed,broker.video,broker.fulfillment_id,recommendations_count,agent_type,is_realtor,first_month,full_name,web_url,mls.0.member.id,mls.0.abbreviation,mls.0.type,mls.0.primary,mls.0.license_number,nick_name,party_id,name,nar_only,background_photo.href,slogan,is_complete_sort_flag,questions_and_answers.question_1.answer_text,questions_and_answers.question_2.answer_text,for_sale_price.last_listing_date,for_sale_price.min,for_sale_price.max,for_sale_price.count,recently_sold.min,recently_sold.max,recently_sold.last_sold_date,recently_sold.count,agent_team_details.is_team_member,office.mls.0.license_number,office.mls.1.member.id,office.mls.1.abbreviation,office.mls.1.type,office.mls.1.primary,office.mls.1.license_number,office.mls.2.member.id,office.mls.2.abbreviation,office.mls.2.type,office.mls.2.primary,office.mls.2.license_number,agent_team_details.team_advertiser_id,designations.0.name,designations.1.name,settings.use_dot_realtor_for_search_engines,specializations.4.name,designations.2.name,designations.3.name,settings.recommendations.tt.user,settings.recommendations.tt.updated,settings.recommendations.tt.linked,specializations.5.name,specializations.6.name,specializations.7.name,phones.1.ext,phones.1.number,phones.1.type,office.phones,office.party_id,first_name,mls_history,last_name,office.phones.1.ext,office.phones.1.number,office.phones.1.type,office.phone_list.phone_2.ext,office.phone_list.phone_2.number,office.phone_list.phone_2.type,office.mls.3.member.id,office.mls.3.abbreviation,office.mls.3.type,office.mls.3.primary,office.mls.4.member.id,office.mls.4.abbreviation,office.mls.4.type,office.mls.4.primary,office.mls.5.member.id,office.mls.5.abbreviation,office.mls.5.type,office.mls.5.primary,broker.address.zip,broker.address.country,broker.address.address_line,broker.address.city,broker.address.state,mls.1.member.id,mls.1.abbreviation,mls.1.type,mls.1.primary,mls.1.license_number,office.mls.3.license_number,office.mls.4.license_number,office.mls.5.license_number,office.mls.6.member.id,office.mls.6.abbreviation,office.mls.6.type,office.mls.6.primary,office.mls.6.license_number,specializations.8.name,specializations.9.name,specializations.10.name,specializations.

In [12]:
for col in df.columns:
    print(col)

state
usps
first_year
user_languages
role
description
phones.0.ext
phones.0.number
phones.0.type
review_count
office.website
office.address.country
office.address.city
office.address.line
office.address.state
office.address.postal_code
office.address.state_code
office.address.line2
office.phones.0.ext
office.phones.0.number
office.phones.0.type
office.feed_licenses
office.photo.href
office.photo.is_zoomed
office.phone_list.phone_1.ext
office.phone_list.phone_1.number
office.phone_list.phone_1.type
office.video
office.nrds_id
office.fulfillment_id
office.licenses
office.mls.0.member.id
office.mls.0.id
office.mls.0.abbreviation
office.mls.0.type
office.mls.0.primary
office.name
office.slogan
zips
title
has_photo
advertiser_id
mls_monetization
designations
agent_rating
href
id
settings.display_ratings
settings.broker_data_feed_opt_out
settings.cfb_wizard_step
settings.terms_of_use
settings.has_dotrealtor
settings.display_price_range
settings.display_listings
settings.show_stream
settings.

In [13]:
df.shape

(940, 898)

In [14]:
df["id"].nunique()

900

In [15]:
import re
import pandas as pd

# ✅ --- Combine all specializations.*.name columns into one clean column ---
spec_cols = [
    c for c in df.columns
    if re.fullmatch(r"specializations\.\d+\.name", c)
]

def combine_specializations(row):
    seen, out = set(), []
    for v in row:
        if pd.notna(v) and v not in seen:
            seen.add(v)
            out.append(str(v).strip())
    return ", ".join(out)

df["specializations_joined"] = df[spec_cols].apply(combine_specializations, axis=1)

# Drop all original specialization columns
df = df.drop(columns=spec_cols, errors="ignore")

# ✅ --- Combine all designations.*.name columns into one clean column ---
desig_cols = [
    c for c in df.columns
    if re.fullmatch(r"designations\.\d+\.name", c)
]

def combine_designations(row):
    seen, out = set(), []
    for v in row:
        if pd.notna(v) and v not in seen:
            seen.add(v)
            out.append(str(v).strip())
    return ", ".join(out)

df["designations_joined"] = df[desig_cols].apply(combine_designations, axis=1)

# Drop all original designation columns
df = df.drop(columns=desig_cols, errors="ignore")

print("✅ Combined all specialization and designation columns.")
print("➡️ Created: 'specializations_joined', 'designations_joined'")
print("➡️ Dropped:", len(spec_cols), "specialization cols and", len(desig_cols), "designation cols.")


✅ Combined all specialization and designation columns.
➡️ Created: 'specializations_joined', 'designations_joined'
➡️ Dropped: 39 specialization cols and 11 designation cols.


In [16]:
df.shape

(940, 850)

In [17]:
for col in df.columns:
    print(col)

state
usps
first_year
user_languages
role
description
phones.0.ext
phones.0.number
phones.0.type
review_count
office.website
office.address.country
office.address.city
office.address.line
office.address.state
office.address.postal_code
office.address.state_code
office.address.line2
office.phones.0.ext
office.phones.0.number
office.phones.0.type
office.feed_licenses
office.photo.href
office.photo.is_zoomed
office.phone_list.phone_1.ext
office.phone_list.phone_1.number
office.phone_list.phone_1.type
office.video
office.nrds_id
office.fulfillment_id
office.licenses
office.mls.0.member.id
office.mls.0.id
office.mls.0.abbreviation
office.mls.0.type
office.mls.0.primary
office.name
office.slogan
zips
title
has_photo
advertiser_id
mls_monetization
designations
agent_rating
href
id
settings.display_ratings
settings.broker_data_feed_opt_out
settings.cfb_wizard_step
settings.terms_of_use
settings.has_dotrealtor
settings.display_price_range
settings.display_listings
settings.show_stream
settings.

In [18]:
df.head(3)

,state,usps,first_year,user_languages,role,description,phones.0.ext,phones.0.number,phones.0.type,review_count,office.website,office.address.country,office.address.city,office.address.line,office.address.state,office.address.postal_code,office.address.state_code,office.address.line2,office.phones.0.ext,office.phones.0.number,office.phones.0.type,office.feed_licenses,office.photo.href,office.photo.is_zoomed,office.phone_list.phone_1.ext,office.phone_list.phone_1.number,office.phone_list.phone_1.type,office.video,office.nrds_id,office.fulfillment_id,office.licenses,office.mls.0.member.id,office.mls.0.id,office.mls.0.abbreviation,office.mls.0.type,office.mls.0.primary,office.name,office.slogan,zips,title,has_photo,advertiser_id,mls_monetization,designations,agent_rating,href,id,settings.display_ratings,settings.broker_data_feed_opt_out,settings.cfb_wizard_step,settings.terms_of_use,settings.has_dotrealtor,settings.display_price_range,settings.display_listings,settings.show_stream,settings.full_access,settings.far_override,settings.loaded_from_sb,settings.unsubscribe.account_notify,settings.unsubscribe.autorecs,settings.unsubscribe.recapprove,settings.new_feature_popup_closed.agent_left_nav_avatar_to_profile,settings.reviews_opt_out.rdc,settings.reviews_opt_out.rs,settings.share_contacts,settings.display_sold_listings,settings.profile_wizard.realsatisfied_opt_out,settings.profile_wizard.tt_opt_out,last_updated,types,address.country,address.city,address.line,address.state,address.postal_code,address.state_code,address.line2,languages,photo.href,photo.is_zoomed,feed_licenses,person_name,nrds_id,broker.accent_color,broker.designations,broker.name,broker.photo.href,broker.photo.is_zoomed,broker.video,broker.fulfillment_id,recommendations_count,agent_type,is_realtor,first_month,full_name,web_url,mls.0.member.id,mls.0.abbreviation,mls.0.type,mls.0.primary,mls.0.license_number,nick_name,party_id,name,nar_only,background_photo.href,slogan,is_complete_sort_flag,questions_and_answers.question_1.answer_text,questions_and_answers.question_2.answer_text,for_sale_price.last_listing_date,for_sale_price.min,for_sale_price.max,for_sale_price.count,recently_sold.min,recently_sold.max,recently_sold.last_sold_date,recently_sold.count,agent_team_details.is_team_member,office.mls.0.license_number,office.mls.1.member.id,office.mls.1.abbreviation,office.mls.1.type,office.mls.1.primary,office.mls.1.license_number,office.mls.2.member.id,office.mls.2.abbreviation,office.mls.2.type,office.mls.2.primary,office.mls.2.license_number,agent_team_details.team_advertiser_id,settings.use_dot_realtor_for_search_engines,settings.recommendations.tt.user,settings.recommendations.tt.updated,settings.recommendations.tt.linked,phones.1.ext,phones.1.number,phones.1.type,office.phones,office.party_id,first_name,mls_history,last_name,office.phones.1.ext,office.phones.1.number,office.phones.1.type,office.phone_list.phone_2.ext,office.phone_list.phone_2.number,office.phone_list.phone_2.type,office.mls.3.member.id,office.mls.3.abbreviation,office.mls.3.type,office.mls.3.primary,office.mls.4.member.id,office.mls.4.abbreviation,office.mls.4.type,office.mls.4.primary,office.mls.5.member.id,office.mls.5.abbreviation,office.mls.5.type,office.mls.5.primary,broker.address.zip,broker.address.country,broker.address.address_line,broker.address.city,broker.address.state,mls.1.member.id,mls.1.abbreviation,mls.1.type,mls.1.primary,mls.1.license_number,office.mls.3.license_number,office.mls.4.license_number,office.mls.5.license_number,office.mls.6.member.id,office.mls.6.abbreviation,office.mls.6.type,office.mls.6.primary,office.mls.6.license_number,video,specializations,phones.0.key,phones.1.key,office.mls,settings.recommendations.realsatisfied.id,settings.recommendations.realsatisfied.user,settings.recommendations.realsatisfied.updated,settings.recommendations.realsatisfied.linked,mls_history.0.member.id,mls_history.0.inactivation_date,mls_history.0.abbreviation,mls_history.0.type,mls_histor

In [19]:
drop_settings = [c for c in df.columns if c.startswith("settings.")]
df = df.drop(columns=drop_settings)

In [20]:
df.shape

(940, 818)

In [21]:
original_df.shape

(940, 14485)

In [22]:
for col in df.columns:
    print(col)

state
usps
first_year
user_languages
role
description
phones.0.ext
phones.0.number
phones.0.type
review_count
office.website
office.address.country
office.address.city
office.address.line
office.address.state
office.address.postal_code
office.address.state_code
office.address.line2
office.phones.0.ext
office.phones.0.number
office.phones.0.type
office.feed_licenses
office.photo.href
office.photo.is_zoomed
office.phone_list.phone_1.ext
office.phone_list.phone_1.number
office.phone_list.phone_1.type
office.video
office.nrds_id
office.fulfillment_id
office.licenses
office.mls.0.member.id
office.mls.0.id
office.mls.0.abbreviation
office.mls.0.type
office.mls.0.primary
office.name
office.slogan
zips
title
has_photo
advertiser_id
mls_monetization
designations
agent_rating
href
id
last_updated
types
address.country
address.city
address.line
address.state
address.postal_code
address.state_code
address.line2
languages
photo.href
photo.is_zoomed
feed_licenses
person_name
nrds_id
broker.accent_co

In [23]:
import re
import pandas as pd

# ---------- helpers ----------
def _truthy(x):
    if pd.isna(x):
        return False
    if isinstance(x, bool):
        return x
    return str(x).strip().lower() in {"true", "t", "1", "yes", "y"}

def _dedup_stable(seq):
    seen, out = set(), []
    for v in seq:
        v = None if pd.isna(v) else str(v).strip()
        if v and v not in seen:
            seen.add(v)
            out.append(v)
    return out

def _summarize_indexed_block(df, base):
    """
    base: 'mls' or 'office.mls'
    Creates: 
      - f'{base}_primary_abbrev'
      - f'{base}_primary_license'
      - f'{base}_abbrevs_joined'
      - f'{base}_licenses_joined'
    Drops all original {base}.* columns afterwards.
    """
    # collect per-index fields present in the dataframe
    field_pat = re.compile(rf"^{re.escape(base)}\.(\d+)\.(member\.id|id|abbreviation|type|primary|license_number)$")
    indices = set()
    fields_for_index = {}  # idx -> {field: colname}
    for col in df.columns:
        m = field_pat.fullmatch(col)
        if not m:
            continue
        idx, field = int(m.group(1)), m.group(2)
        indices.add(idx)
        fields_for_index.setdefault(idx, {})[field] = col
    indices = sorted(indices)

    # bail early if nothing to do
    if not indices:
        return df

    def _row_summarize(row):
        # gather all items
        items = []
        for i in indices:
            f = fields_for_index.get(i, {})
            abbr = row.get(f.get("abbreviation")) if f.get("abbreviation") else None
            lic  = row.get(f.get("license_number")) if f.get("license_number") else None
            prim = row.get(f.get("primary")) if f.get("primary") else None
            items.append({
                "abbr": None if pd.isna(abbr) else str(abbr).strip(),
                "lic":  None if pd.isna(lic) else str(lic).strip(),
                "prim": _truthy(prim)
            })

        # choose primary first; else first non-null
        primary = next((x for x in items if x["prim"] and (x["abbr"] or x["lic"])), None)
        if primary is None:
            primary = next((x for x in items if x["abbr"] or x["lic"]), None) or {"abbr": None, "lic": None}

        # build deduped joined lists (keep order by index)
        abbrevs = _dedup_stable([x["abbr"] for x in items if x["abbr"]])
        licenses = _dedup_stable([x["lic"] for x in items if x["lic"]])

        return pd.Series({
            f"{base}_primary_abbrev": primary["abbr"],
            f"{base}_primary_license": primary["lic"],
            f"{base}_abbrevs_joined": ", ".join(abbrevs),
            f"{base}_licenses_joined": ", ".join(licenses),
        })

    newcols = df.apply(_row_summarize, axis=1)
    df = pd.concat([df, newcols], axis=1)

    # drop originals
    to_drop = [c for c in df.columns if c.startswith(base + ".")]
    df = df.drop(columns=to_drop, errors="ignore")
    # also drop the root object column if it exists (some exports include it)
    if base == "office.mls" and "office.mls" in df.columns:
        df = df.drop(columns=["office.mls"])
    if base == "mls" and "mls" in df.columns:
        df = df.drop(columns=["mls"])
    return df


def _summarize_mls_history(df):
    """
    Summarize mls_history.*.* into:
      - mls_history_joined      (e.g., "ABR|LIC|inactive=2024-05-10, ...")
      - mls_history_last_inactivated_abbrev
      - mls_history_last_inactivated_date
    Then drop all mls_history.* original columns.
    """
    pat = re.compile(r"^mls_history\.(\d+)\.(member\.id|inactivation_date|abbreviation|type|primary|license_number)$")
    indices = set()
    fields_for_index = {}
    for col in df.columns:
        m = pat.fullmatch(col)
        if not m:
            continue
        idx, field = int(m.group(1)), m.group(2)
        indices.add(idx)
        fields_for_index.setdefault(idx, {})[field] = col
    indices = sorted(indices)

    if not indices:
        return df

    def _row_hist(row):
        rows = []
        for i in indices:
            f = fields_for_index.get(i, {})
            abbr = row.get(f.get("abbreviation")) if f.get("abbreviation") else None
            lic  = row.get(f.get("license_number")) if f.get("license_number") else None
            inact = row.get(f.get("inactivation_date")) if f.get("inactivation_date") else None

            abbr = None if pd.isna(abbr) else str(abbr).strip()
            lic  = None if pd.isna(lic)  else str(lic).strip()
            inact_s = None if pd.isna(inact) else str(inact).strip()

            # build a compact token
            parts = []
            if abbr: parts.append(abbr)
            if lic: parts.append(lic)
            token = "|".join(parts) if parts else None
            if token:
                if inact_s:
                    token = f"{token}|inactive={inact_s}"
                rows.append((inact_s, token, abbr))

        # sort by inactivation_date desc when it looks like a date; else keep index order
        def _key(t):
            date_str = t[0]
            try:
                return pd.to_datetime(date_str)
            except Exception:
                return pd.NaT
        rows_sorted = sorted(rows, key=_key, reverse=True)

        joined = ", ".join(_dedup_stable([t[1] for t in rows_sorted if t[1]]))
        last_abbrev = rows_sorted[0][2] if rows_sorted else None
        last_date = rows_sorted[0][0] if rows_sorted else None

        return pd.Series({
            "mls_history_joined": joined,
            "mls_history_last_inactivated_abbrev": last_abbrev,
            "mls_history_last_inactivated_date": last_date,
        })

    newcols = df.apply(_row_hist, axis=1)
    df = pd.concat([df, newcols], axis=1)

    # drop originals
    drop_hist = [c for c in df.columns if c.startswith("mls_history.")]
    df = df.drop(columns=drop_hist, errors="ignore")
    if "mls_history" in df.columns:
        df = df.drop(columns=["mls_history"])
    return df


# ---------- run the summaries ----------
# 1) agent-level MLS (mls.*)
df = _summarize_indexed_block(df, base="mls")

# 2) office-level MLS (office.mls.*)
df = _summarize_indexed_block(df, base="office.mls")

# 3) historical MLS (mls_history.*)
df = _summarize_mls_history(df)


print("✅ MLS columns summarized.")
print("  • Created columns:",
      [c for c in df.columns if c.endswith("_primary_abbrev") or c.endswith("_primary_license")
       or c.endswith("_abbrevs_joined") or c.endswith("_licenses_joined")
       or c.startswith("mls_history_")])


✅ MLS columns summarized.
  • Created columns: ['mls_primary_abbrev', 'mls_primary_license', 'mls_abbrevs_joined', 'mls_licenses_joined', 'office.mls_primary_abbrev', 'office.mls_primary_license', 'office.mls_abbrevs_joined', 'office.mls_licenses_joined', 'mls_history_joined', 'mls_history_last_inactivated_abbrev', 'mls_history_last_inactivated_date']


In [24]:
df.shape

(940, 175)

In [25]:
df.head(3)

,state,usps,first_year,user_languages,role,description,phones.0.ext,phones.0.number,phones.0.type,review_count,office.website,office.address.country,office.address.city,office.address.line,office.address.state,office.address.postal_code,office.address.state_code,office.address.line2,office.phones.0.ext,office.phones.0.number,office.phones.0.type,office.feed_licenses,office.photo.href,office.photo.is_zoomed,office.phone_list.phone_1.ext,office.phone_list.phone_1.number,office.phone_list.phone_1.type,office.video,office.nrds_id,office.fulfillment_id,office.licenses,office.name,office.slogan,zips,title,has_photo,advertiser_id,mls_monetization,designations,agent_rating,href,id,last_updated,types,address.country,address.city,address.line,address.state,address.postal_code,address.state_code,address.line2,languages,photo.href,photo.is_zoomed,feed_licenses,person_name,nrds_id,broker.accent_color,broker.designations,broker.name,broker.photo.href,broker.photo.is_zoomed,broker.video,broker.fulfillment_id,recommendations_count,agent_type,is_realtor,first_month,full_name,web_url,nick_name,party_id,name,nar_only,background_photo.href,slogan,is_complete_sort_flag,questions_and_answers.question_1.answer_text,questions_and_answers.question_2.answer_text,for_sale_price.last_listing_date,for_sale_price.min,for_sale_price.max,for_sale_price.count,recently_sold.min,recently_sold.max,recently_sold.last_sold_date,recently_sold.count,agent_team_details.is_team_member,agent_team_details.team_advertiser_id,phones.1.ext,phones.1.number,phones.1.type,office.phones,office.party_id,first_name,last_name,office.phones.1.ext,office.phones.1.number,office.phones.1.type,office.phone_list.phone_2.ext,office.phone_list.phone_2.number,office.phone_list.phone_2.type,broker.address.zip,broker.address.country,broker.address.address_line,broker.address.city,broker.address.state,video,specializations,phones.0.key,phones.1.key,phones.2.ext,phones.2.number,phones.2.type,office.licenses.0.country,office.licenses.0.state_code,office.licenses.0.license_number,address,office.phones.2.ext,office.phones.2.number,office.phones.2.type,office.phone_list.phone_3.ext,office.phone_list.phone_3.number,office.phone_list.phone_3.type,questions_and_answers.question_3.answer_text,questions_and_answers.question_4.answer_text,office.licenses.1.country,office.licenses.1.state_code,office.licenses.1.license_number,phones.2.key,phones.3.ext,phones.3.number,phones.3.type,feed_licenses.0.country,feed_licenses.0.state_code,feed_licenses.0.license_number,office.licenses.2.country,office.licenses.2.state_code,office.licenses.2.license_number,office.licenses.3.country,office.licenses.3.state_code,office.licenses.3.license_number,office.licenses.4.country,office.licenses.4.state_code,office.licenses.4.license_number,office.licenses.5.country,office.licenses.5.state_code,office.licenses.5.license_number,office.phone_list.phone_4.type,office.phone_list.phone_4.number,office.phone_list.phone_4.ext,services.seller,services.buyer,office.franchise_name,phones,phones.3.key,office.phones.3.ext,office.phones.3.number,office.phones.3.type,areas_of_business,marketing_area_cities_as_city_state,served_areas_as_name_state,specializations_joined,designations_joined,mls_primary_abbrev,mls_primary_license,mls_abbrevs_joined,mls_licenses_joined,office.mls_primary_abbrev,office.mls_primary_license,office.mls_abbrevs_joined,office.mls_licenses_joined,mls_history_joined,mls_history_last_inactivated_abbrev,mls_history_last_inactivated_date
0,Alaska,AK,2018.0,NaN,agent,"Welcome to my website! I’m thrilled you’re here. Writing about myself isn’t my favorite thing, but here’s a bit about me.\n\nIn 2015, my husband and I traded the warmth of Gainesville, Florida for the rugged beauty of Wasilla, Alaska. Go Gators! With our youngest daughter off to college, we were ready for a new adventure, and Alaska has been just that.\n\nWe have three daughters and a wonderful grandson who loves visiting us in Alaska. Our four dogs—a loya

In [26]:
for col in df.columns:
    print(col)

state
usps
first_year
user_languages
role
description
phones.0.ext
phones.0.number
phones.0.type
review_count
office.website
office.address.country
office.address.city
office.address.line
office.address.state
office.address.postal_code
office.address.state_code
office.address.line2
office.phones.0.ext
office.phones.0.number
office.phones.0.type
office.feed_licenses
office.photo.href
office.photo.is_zoomed
office.phone_list.phone_1.ext
office.phone_list.phone_1.number
office.phone_list.phone_1.type
office.video
office.nrds_id
office.fulfillment_id
office.licenses
office.name
office.slogan
zips
title
has_photo
advertiser_id
mls_monetization
designations
agent_rating
href
id
last_updated
types
address.country
address.city
address.line
address.state
address.postal_code
address.state_code
address.line2
languages
photo.href
photo.is_zoomed
feed_licenses
person_name
nrds_id
broker.accent_color
broker.designations
broker.name
broker.photo.href
broker.photo.is_zoomed
broker.video
broker.fulfill

In [27]:
import re
import pandas as pd

# ---------- helpers ----------
def _dedup_stable_strs(values):
    seen, out = set(), []
    for v in values:
        if v is None or (isinstance(v, float) and pd.isna(v)): 
            continue
        s = str(v).strip()
        if not s or s in seen:
            continue
        seen.add(s)
        out.append(s)
    return out

def _get(df, col):
    return df[col] if col in df.columns else pd.Series([None]*len(df), index=df.index)

# ---------- OFFICE ADDRESS: join into one printable string ----------
addr_cols = {
    "line": "office.address.line",
    "line2": "office.address.line2",
    "city": "office.address.city",
    "state_code": "office.address.state_code",
    "state": "office.address.state",              # fallback if no state_code
    "postal": "office.address.postal_code",
    "country": "office.address.country",
}
parts = []
for key, col in addr_cols.items():
    if col in df.columns:
        parts.append(_get(df, col).astype("string"))
    else:
        parts.append(pd.Series([pd.NA]*len(df), index=df.index, dtype="string"))

addr_line, addr_line2, addr_city, addr_state_code, addr_state, addr_postal, addr_country = parts

office_state_pref = addr_state_code.fillna(addr_state)  # prefer state_code when present

def _fmt_addr(row):
    bits = _dedup_stable_strs([
        row["line"],
        row["line2"],
        row["city"],
        # "ST ZIP" if both, else whichever exists
        " ".join(_dedup_stable_strs([row["state_pref"], row["postal"]])),
        row["country"]
    ])
    return ", ".join([b for b in bits if b])

df["office_address_joined"] = pd.DataFrame({
    "line": addr_line,
    "line2": addr_line2,
    "city": addr_city,
    "state_pref": office_state_pref,
    "postal": addr_postal,
    "country": addr_country
}).apply(_fmt_addr, axis=1)

# ---------- OFFICE PHONES: unify office.phones.* and office.phone_list.* ----------
# Gather all indexed phone columns of both styles
# Patterns:
#   office.phones.<idx>.(number|ext|type)
#   office.phone_list.phone_<idx>.(number|ext|type)
phones_map = {}  # idx -> {number, ext, type}
phone_pat1 = re.compile(r"^office\.phones\.(\d+)\.(number|ext|type)$")
phone_pat2 = re.compile(r"^office\.phone_list\.phone_(\d+)\.(number|ext|type)$")

for col in df.columns:
    m1 = phone_pat1.match(col)
    m2 = phone_pat2.match(col)
    if m1:
        i, field = int(m1.group(1)), m1.group(2)
        phones_map.setdefault(i, {})[field] = col
    elif m2:
        i, field = int(m2.group(1)), m2.group(2)
        phones_map.setdefault(i, {})[field] = col

idxs = sorted(phones_map.keys())

def _row_office_phones(row):
    items = []
    for i in idxs:
        cols = phones_map[i]
        num = row.get(cols.get("number")) if "number" in cols else None
        ext = row.get(cols.get("ext")) if "ext" in cols else None
        typ = row.get(cols.get("type")) if "type" in cols else None

        num = None if pd.isna(num) else str(num).strip()
        ext = None if pd.isna(ext) else str(ext).strip()
        typ = None if pd.isna(typ) else str(typ).strip()

        if num:
            disp = num
            if ext:
                disp = f"{disp} x{ext}"
            if typ:
                disp = f"{disp} ({typ})"
            items.append({"num": num, "type": typ, "display": disp})

    # primary = first of type 'Office' else first
    primary = next((x for x in items if (x["type"] or "").lower() == "office"), items[0] if items else {"num": None, "type": None})
    joined = ", ".join(_dedup_stable_strs([x["display"] for x in items]))

    return pd.Series({
        "office_phone_primary_number": primary["num"],
        "office_phone_primary_type": primary["type"],
        "office_phone_numbers_joined": joined
    })

if idxs:
    df = pd.concat([df, df.apply(_row_office_phones, axis=1)], axis=1)
else:
    # still create empty cols for consistency
    for c in ["office_phone_primary_number", "office_phone_primary_type", "office_phone_numbers_joined"]:
        if c not in df.columns:
            df[c] = None

# ---------- OFFICE LICENSES: compress multi-index blocks ----------
# Patterns:
#   office.licenses.<idx>.(country|state_code|license_number)
lic_pat = re.compile(r"^office\.licenses\.(\d+)\.(country|state_code|license_number)$")
lic_map = {}  # idx -> fields
for col in df.columns:
    m = lic_pat.match(col)
    if m:
        i, field = int(m.group(1)), m.group(2)
        lic_map.setdefault(i, {})[field] = col

lic_idxs = sorted(lic_map.keys())

def _row_office_licenses(row):
    tokens = []
    states = []
    for i in lic_idxs:
        rec = lic_map[i]
        country = row.get(rec.get("country")) if "country" in rec else None
        state = row.get(rec.get("state_code")) if "state_code" in rec else None
        lic   = row.get(rec.get("license_number")) if "license_number" in rec else None

        country = None if pd.isna(country) else str(country).strip()
        state   = None if pd.isna(state)   else str(state).strip()
        lic     = None if pd.isna(lic)     else str(lic).strip()

        # prefer STATE:LICENSE, else COUNTRY:LICENSE, else just LICENSE
        if state and lic:
            tokens.append(f"{state}:{lic}")
            states.append(state)
        elif country and lic:
            tokens.append(f"{country}:{lic}")
        elif lic:
            tokens.append(lic)

    tokens = _dedup_stable_strs(tokens)
    states = _dedup_stable_strs(states)
    return pd.Series({
        "office_licenses_joined": ", ".join(tokens),
        "office_license_states_joined": ", ".join(states),
    })

if lic_idxs:
    df = pd.concat([df, df.apply(_row_office_licenses, axis=1)], axis=1)
else:
    for c in ["office_licenses_joined", "office_license_states_joined"]:
        if c not in df.columns:
            df[c] = None

# ---------- DROP noisy original office subtrees that are now summarized ----------
drop_prefixes = [
    "office.address.", 
    "office.phones.", 
    "office.phone_list.", 
    "office.licenses."
]
to_drop = [c for c in df.columns if any(c.startswith(p) for p in drop_prefixes)]
# keep a short allowlist of single fields you might still want (edit if needed)
keep_exact = {
    "office.website",
    "office.photo.href", "office.photo.is_zoomed",
    "office.video",
    "office.name", "office.slogan",
    "office.feed_licenses",   # if you still want raw feed trail
    "office.franchise_name",
    "office.nrds_id", "office.fulfillment_id",
    "office.party_id",
}
to_drop = [c for c in to_drop if c not in keep_exact]
df = df.drop(columns=to_drop, errors="ignore")

# also drop any dangling root objects if present
for root in ["office.phones", "office.licenses"]:
    if root in df.columns:
        df = df.drop(columns=[root])

print("✅ Office fields cleaned:",
      ["office_address_joined",
       "office_phone_primary_number", "office_phone_primary_type", "office_phone_numbers_joined",
       "office_licenses_joined", "office_license_states_joined"])


✅ Office fields cleaned: ['office_address_joined', 'office_phone_primary_number', 'office_phone_primary_type', 'office_phone_numbers_joined', 'office_licenses_joined', 'office_license_states_joined']


In [31]:
df.shape

(940, 130)

In [32]:
for col in df.columns:
    print(col)

state
usps
first_year
user_languages
role
description
phones.0.ext
phones.0.number
phones.0.type
review_count
office.website
office.feed_licenses
office.photo.href
office.photo.is_zoomed
office.video
office.nrds_id
office.fulfillment_id
office.name
office.slogan
zips
title
has_photo
advertiser_id
mls_monetization
designations
agent_rating
href
id
last_updated
types
address.country
address.city
address.line
address.state
address.postal_code
address.state_code
address.line2
languages
photo.href
photo.is_zoomed
feed_licenses
person_name
nrds_id
broker.accent_color
broker.designations
broker.name
broker.photo.href
broker.photo.is_zoomed
broker.video
broker.fulfillment_id
recommendations_count
agent_type
is_realtor
first_month
full_name
web_url
nick_name
party_id
name
nar_only
background_photo.href
slogan
is_complete_sort_flag
questions_and_answers.question_1.answer_text
questions_and_answers.question_2.answer_text
for_sale_price.last_listing_date
for_sale_price.min
for_sale_price.max
for_s

In [33]:
import re
import pandas as pd

# ---------- helpers ----------
def _dedup_stable(values):
    seen, out = set(), []
    for v in values:
        if v is None or (isinstance(v, float) and pd.isna(v)):
            continue
        s = str(v).strip()
        if not s or s in seen:
            continue
        seen.add(s)
        out.append(s)
    return out

# ---------- collect phones.* columns ----------
# patterns like: phones.0.number / phones.0.ext / phones.0.type / phones.0.key
ph_pat = re.compile(r"^phones\.(\d+)\.(number|ext|type|key)$")

phones_map = {}  # idx -> dict(number=colname, ext=..., type=..., key=...)
for col in df.columns:
    m = ph_pat.match(col)
    if m:
        i, field = int(m.group(1)), m.group(2)
        phones_map.setdefault(i, {})[field] = col

idxs = sorted(phones_map.keys())

def _row_person_phones(row):
    items = []  # list of dicts: {"num","ext","type","key","disp"}
    for i in idxs:
        cols = phones_map[i]
        num = row.get(cols.get("number")) if "number" in cols else None
        ext = row.get(cols.get("ext")) if "ext" in cols else None
        typ = row.get(cols.get("type")) if "type" in cols else None
        key = row.get(cols.get("key")) if "key" in cols else None

        num = None if pd.isna(num) else str(num).strip()
        ext = None if pd.isna(ext) else str(ext).strip()
        typ = None if pd.isna(typ) else str(typ).strip()
        key = None if pd.isna(key) else str(key).strip()

        if not num:
            continue

        disp = num
        if ext:
            disp = f"{disp} x{ext}"
        if typ:
            disp = f"{disp} ({typ})"

        items.append({"num": num, "ext": ext, "type": typ, "key": key, "disp": disp})

    # choose primary: prefer Mobile, then Office, else first
    def _is_type(x, want):
        return (x.get("type") or "").strip().lower() == want
    primary = None
    for want in ("mobile", "office"):
        primary = next((x for x in items if _is_type(x, want)), None)
        if primary:
            break
    if primary is None:
        primary = items[0] if items else {"num": None, "type": None}

    numbers_joined = ", ".join(_dedup_stable([x["num"] for x in items]))
    display_joined = ", ".join(_dedup_stable([x["disp"] for x in items]))

    return pd.Series({
        "phone_primary_number": primary.get("num"),
        "phone_primary_type": primary.get("type"),
        "phone_numbers_joined": numbers_joined,
        "phone_display_joined": display_joined,
    })

# apply and attach (even if no phones.* exist, ensure columns are present)
if idxs:
    df = pd.concat([df, df.apply(_row_person_phones, axis=1)], axis=1)
else:
    for c in ["phone_primary_number", "phone_primary_type", "phone_numbers_joined", "phone_display_joined"]:
        if c not in df.columns:
            df[c] = None

# ---------- drop original phones.* noise ----------
drop_cols = [c for c in df.columns if c.startswith("phones.")]
if "phones" in df.columns:
    drop_cols.append("phones")
df = df.drop(columns=drop_cols, errors="ignore")


In [34]:
df.shape

(940, 117)

In [35]:
for col in df.columns:
    print(col)

state
usps
first_year
user_languages
role
description
review_count
office.website
office.feed_licenses
office.photo.href
office.photo.is_zoomed
office.video
office.nrds_id
office.fulfillment_id
office.name
office.slogan
zips
title
has_photo
advertiser_id
mls_monetization
designations
agent_rating
href
id
last_updated
types
address.country
address.city
address.line
address.state
address.postal_code
address.state_code
address.line2
languages
photo.href
photo.is_zoomed
feed_licenses
person_name
nrds_id
broker.accent_color
broker.designations
broker.name
broker.photo.href
broker.photo.is_zoomed
broker.video
broker.fulfillment_id
recommendations_count
agent_type
is_realtor
first_month
full_name
web_url
nick_name
party_id
name
nar_only
background_photo.href
slogan
is_complete_sort_flag
questions_and_answers.question_1.answer_text
questions_and_answers.question_2.answer_text
for_sale_price.last_listing_date
for_sale_price.min
for_sale_price.max
for_sale_price.count
recently_sold.min
recently_

In [38]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Now show the full null summary
df.isnull().sum()

state                                             0
usps                                              0
first_year                                        2
user_languages                                  805
role                                              0
description                                     166
review_count                                      0
office.website                                  334
office.feed_licenses                            940
office.photo.href                               518
office.photo.is_zoomed                          940
office.video                                    921
office.nrds_id                                  586
office.fulfillment_id                             0
office.name                                      15
office.slogan                                   721
zips                                             12
title                                           105
has_photo                                         0
advertiser_i

In [41]:
df.duplicated().sum()

0

In [42]:
df = df.drop_duplicates(subset=["id"], keep="first").reset_index(drop=True)

In [43]:
df.shape

(900, 117)

In [44]:
# ✅ Export the cleaned dataframe
output_path = "agents_full_details_out.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ Exported cleaned agent dataset to: {output_path}")
print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")


✅ Exported cleaned agent dataset to: agents_full_details_out.csv
Shape: 900 rows × 117 columns


In [45]:
# --- define columns to keep ---
keep_cols = [
    # basic identifiers
    "id", "full_name", "first_name", "last_name", "state", "usps",
    "first_year", "first_month", "role", "agent_type", "is_realtor",
    "person_name", "party_id", "advertiser_id",
    
    # performance & stats
    "review_count", "agent_rating", "recommendations_count",
    "for_sale_price.min", "for_sale_price.max", "for_sale_price.count",
    "recently_sold.min", "recently_sold.max", "recently_sold.count",
    "for_sale_price.last_listing_date" , "recently_sold.last_sold_date",            
    "mls_primary_abbrev", "mls_abbrevs_joined", "mls_licenses_joined",
    "mls_history_joined", "mls_history_last_inactivated_abbrev",
    "mls_history_last_inactivated_date",

    # expertise / areas
    "specializations_joined", "designations_joined", "user_languages",
    "served_areas_as_name_state", "marketing_area_cities_as_city_state",
    "zips", "areas_of_business", "services.seller", "services.buyer",

    # office & broker
    "office.name", "office.fulfillment_id", "office.franchise_name",
    "office_address_joined", "office_phone_primary_number",
    "office_phone_primary_type", "office_phone_numbers_joined",
    "office_licenses_joined", "office_license_states_joined",
    "broker.name", "broker.fulfillment_id",
    
    # contact / links
    "phone_primary_number", "phone_primary_type",
    "phone_numbers_joined", "phone_display_joined",
    "web_url", "photo.href","href",
]

# --- actually filter dataframe ---
existing_cols = [c for c in keep_cols if c in df.columns]
df_clean = df[existing_cols].copy()

# --- show summary ---
print("✅ Created df_clean with:")
print(f"  → {df_clean.shape[0]} rows × {df_clean.shape[1]} columns")
print("Dropped:", len(df.columns) - len(existing_cols), "irrelevant columns")

# --- quick null overview of what's left ---
display(df_clean.isnull().sum().sort_values(ascending=False).head(20))


✅ Created df_clean with:
  → 900 rows × 58 columns
Dropped: 59 irrelevant columns


services.buyer                         898
services.seller                        898
areas_of_business                      898
office.franchise_name                  894
user_languages                         770
mls_history_last_inactivated_abbrev    767
mls_history_last_inactivated_date      767
last_name                              350
first_name                             348
agent_type                             221
broker.name                            154
recently_sold.last_sold_date           120
href                                   112
office_phone_primary_type               92
office_phone_primary_number             78
photo.href                              22
office.name                             15
phone_primary_type                      14
phone_primary_number                    14
zips                                    12
dtype: int64

In [46]:
drop_cols = [
    "services.buyer",
    "services.seller",
    "areas_of_business",
    "office.franchise_name",
]

df_clean = df_clean.drop(columns=[c for c in drop_cols if c in df_clean.columns], errors="ignore")


In [47]:
df_clean.shape

(900, 54)

In [48]:
for col in df_clean.columns:
    print(col)

id
full_name
first_name
last_name
state
usps
first_year
first_month
role
agent_type
is_realtor
person_name
party_id
advertiser_id
review_count
agent_rating
recommendations_count
for_sale_price.min
for_sale_price.max
for_sale_price.count
recently_sold.min
recently_sold.max
recently_sold.count
for_sale_price.last_listing_date
recently_sold.last_sold_date
mls_primary_abbrev
mls_abbrevs_joined
mls_licenses_joined
mls_history_joined
mls_history_last_inactivated_abbrev
mls_history_last_inactivated_date
specializations_joined
designations_joined
user_languages
served_areas_as_name_state
marketing_area_cities_as_city_state
zips
office.name
office.fulfillment_id
office_address_joined
office_phone_primary_number
office_phone_primary_type
office_phone_numbers_joined
office_licenses_joined
office_license_states_joined
broker.name
broker.fulfillment_id
phone_primary_number
phone_primary_type
phone_numbers_joined
phone_display_joined
web_url
photo.href
href


In [49]:
output_path = "agents_details_clean.csv"
df_clean.to_csv(output_path, index=False, encoding="utf-8-sig")


In [51]:
df_clean["advertiser_id"].isnull().sum()

0

In [52]:
# --- Export only advertiser_id column ---
df_clean[["advertiser_id"]].to_csv("agent_advertiser_ids.csv", index=False, encoding="utf-8-sig")

print("✅ Exported single-column CSV → agent_advertiser_ids.csv")
print(f"📊 Rows: {len(df_clean)} | Columns: 1")


✅ Exported single-column CSV → agent_advertiser_ids.csv
📊 Rows: 900 | Columns: 1


In [53]:
# Count rows where review_count > 0
non_zero_reviews = df_clean[df_clean["review_count"].fillna(0).astype(int) > 0]

print(f"✅ Agents with non-zero reviews: {len(non_zero_reviews)} / {len(df_clean)} total")


✅ Agents with non-zero reviews: 431 / 900 total


In [54]:
df_clean["advertiser_id"].duplicated().sum()

0

In [55]:
# Filter where review_count > 0
non_zero_df = df_clean[df_clean["review_count"].fillna(0).astype(int) > 0]

# Keep only advertiser_id column and drop duplicates
non_zero_ids = non_zero_df[["advertiser_id"]].drop_duplicates().reset_index(drop=True)

# Save to new CSV
non_zero_ids.to_csv("non_zero_review_agent_advertiser_ids.csv", index=False, encoding="utf-8-sig")

print(f"✅ Saved {len(non_zero_ids)} advertiser IDs with non-zero reviews → non_zero_review_agent_advertiser_ids.csv")

✅ Saved 431 advertiser IDs with non-zero reviews → non_zero_review_agent_advertiser_ids.csv


In [56]:


# filter: review_count == 0 but recommendations_count > 0
mask = (df_clean["review_count"] == 0) & (df_clean["recommendations_count"] > 0)
subset = df_clean[mask]

# show counts
print(f"✅ Agents with 0 reviews but >0 recommendations: {len(subset)} / {len(df)} total")

✅ Agents with 0 reviews but >0 recommendations: 98 / 900 total


In [57]:
# Filter condition: review_count == 0 & recommendations_count > 0
filtered_df = df[(df["review_count"] == 0) & (df["recommendations_count"] > 0)]

# Extract only advertiser_id, remove duplicates
advertiser_ids = filtered_df[["advertiser_id"]].drop_duplicates().reset_index(drop=True)

# Save to CSV
advertiser_ids.to_csv("review_zero_recommendation_nonzero_advertiser_ids.csv", index=False, encoding="utf-8-sig")

print(f"✅ Saved {len(advertiser_ids)} advertiser IDs → review_zero_recommendation_nonzero_advertiser_ids.csv")

✅ Saved 98 advertiser IDs → review_zero_recommendation_nonzero_advertiser_ids.csv
